In [ ]:
import json
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import torch

import utils

In [ ]:
def get_text_vectors(data, model, tokenizer):
    """
    データセットからテキストを抽出し、トークナイザーとモデルを使用してテキストのベクトルを生成します。

    Args:
    data (dict): キーとテキストのリストを含む辞書。
    model (transformers.PreTrainedModel): テキストをベクトルに変換するためのモデル。
    tokenizer (transformers.PreTrainedTokenizer): テキストをトークンに変換するためのトークナイザー。

    Returns:
    dict: 各キーに対応するテキストベクトルの辞書。
    """
    text_vectors = {}
    for key in data.keys():
        texts = [entry['instruction'] for entry in data[key]]
        encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            model_output = model(**encoded_input)
        # Get the mean of the last hidden states to use as the sentence vector
        sentence_vectors = model_output.last_hidden_state.mean(dim=1).numpy()
        text_vectors[key] = sentence_vectors
    return text_vectors

def plot_dimension_reduction(text_vectors, data, method='tsne'):
    """
    t-SNEまたはPCAを使用して、各キーごとにテキストベクトルを2次元空間に別々のプロットとして表示します。

    Args:
    text_vectors (dict): テキストベクトルの辞書。
    data (dict): 元のデータセット。
    method (str): 使用する次元削減の方法 ('tsne' または 'pca')。
    """
    if method == 'tsne':
        # perplexityの値をサンプル数に応じて調整
        min_samples = min(len(vectors) for vectors in text_vectors.values())
        perplexity_value = min(10, max(5, min_samples - 1))  # 5と10の間で、かつサンプル数-1以下
        reducer = TSNE(n_components=2, random_state=42, perplexity=perplexity_value)
    elif method == 'pca':
        from sklearn.decomposition import PCA
        reducer = PCA(n_components=2)
    else:
        raise ValueError("無効な次元削減方法が指定されました。'tsne' または 'pca' を選択してください。")

    colors = plt.cm.get_cmap('tab20', len(data.keys()))

    for idx, (key, vectors) in enumerate(text_vectors.items()):
        plt.figure()
        transformed_vectors = reducer.fit_transform(vectors)
        ids = [entry['id'] for entry in data[key]]
        unique_ids = list(set(ids))
        color_map = {uid: colors(i) for i, uid in enumerate(unique_ids)}
        label_added = set()

        for i, vec in enumerate(transformed_vectors):
            if ids[i] not in label_added:
                plt.scatter(*vec, color=color_map[ids[i]], label=ids[i])
                label_added.add(ids[i])
            else:
                plt.scatter(*vec, color=color_map[ids[i]])

        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title(f'{method.upper()} plot of Instructions for {key}')
        plt.xlabel('Component 1')
        plt.ylabel('Component 2')
        plt.show()

In [ ]:
file_path = "./test_output/test_generated_for_check_diversity_v1.1.json"
data = utils.load_json_file(file_path)

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large")

text_vectors = get_text_vectors(data, model, tokenizer)

In [ ]:
plot_dimension_reduction(text_vectors, data, method="pca")

In [ ]:
plot_dimension_reduction(text_vectors, data, method="tsne")

In [ ]:
file_path = "./test_output/test_generated_for_check_diversity_v1.0.json"
data = utils.load_json_file(file_path)

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large")

text_vectors = get_text_vectors(data, model, tokenizer)

In [ ]:
plot_dimension_reduction(text_vectors, data, method="pca")

In [ ]:
plot_dimension_reduction(text_vectors, data, method="tsne")

In [ ]:
file_path = "./test_output/test_generated_for_check_diversity_v1.2.json"
data = utils.load_json_file(file_path)

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large")

text_vectors = get_text_vectors(data, model, tokenizer)

In [ ]:
plot_dimension_reduction(text_vectors, data, method="pca")

In [ ]:
plot_dimension_reduction(text_vectors, data, method="tsne")

In [ ]:
file_path = "./test_output/test_generated_for_check_diversity_v1.3.json"
data = utils.load_json_file(file_path)

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large")

text_vectors = get_text_vectors(data, model, tokenizer)

In [ ]:
plot_dimension_reduction(text_vectors, data, method="pca")

In [ ]:
plot_dimension_reduction(text_vectors, data, method="tsne")